In [ ]:
%matplotlib inline
import datetime

import numpy
import pandas
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats

import common2
from common2 import load_data

In [ ]:
# Earliest date that there is sufficient data for all states, including MA
EARLIEST_DATE = pandas.Period('2020-03-13', freq='D')

# Set a latest date when the most recent days have garbage (like on or after holidays)
LATEST_DATE = pandas.Period(datetime.date.today(), freq='D')
LATEST_DATE = pandas.Period('2021-05-22', freq='D')

STATS_LAG = 14
RATIO_DAYS = 14
MIN_SHIFT = 4
MAX_SHIFT = 12

In [ ]:
latest_date, meta, all_stats, cdc_stats, hosp_stats = load_data(EARLIEST_DATE, LATEST_DATE)

In [ ]:
all_stats.tail()

In [ ]:
STATES = all_stats.reset_index().ST.unique()
STATES

In [ ]:
# DOD_STATES

In [ ]:
EXCLUDED = ['AK', 'HI', 'VT', ]

DOD_META = [
    ('AL', 7, 28, True),   ('AR', 5, 42, False),  ('AZ', 6, 28, True),   ('CA', 4, 28, False),
    ('CO', 10, 28, False), ('CT', 5, 20, True),   ('DC', 7, 28, False),  ('DE', 4, 28, True),
    ('FL', 7, 24, True),   ('GA', 8, 30, True),   ('IA', 8, 30, True),   ('ID', 4, 28, False),
    ('IL', 11, 28, False), ('IN', 5, 30, True),   ('KS', 5, 35, True),   ('KY', 8, 28, False),
    ('LA', 8, 28, False),  ('MA', 11, 7, True),   ('MD', 10, 32, False), ('ME', 4, 28, False),
    ('MI', 10, 21, True),  ('MN', 11, 28, False), ('MO', 5, 35, True),   ('MS', 8, 25, True),
    ('MT', 10, 28, False), ('NC', 6, 28, True),   ('ND', 6, 25, True),   ('NE', 7, 28, False),
    ('NH', 10, 35, False), ('NJ', 10, 28, True),  ('NM', 5, 28, False),  ('NV', 10, 24, True),
    ('NY', 7, 28, False),  ('OH', 9, 21, True),   ('OK', 4, 35, False),  ('OR', 5, 35, False),
    ('PA', 8, 30, True),   ('RI', 0, 20, True),   ('SC', 9, 35, True),   ('SD', 12, 38, True),
    ('TN', 9, 25, True),   ('TX', 4, 28, True),   ('UT', 2, 32, False),  ('VA', 5, 28, True),
    ('WA', 4, 28, False),  ('WI', 4, 28, False),  ('WV', 4, 28, False),  ('WY', 12, 28, False),
]

idx = 24

st, hosp_lag, max_lag, __ = DOD_META[idx]  # MO 9, OH 15, VA 22
min_date = '2020-08-01'
# idx = (idx + 1) % len(DOD_META)
min_date, max_date = pandas.Period(min_date, freq='D'), LATEST_DATE - max_lag
stats_max_date = max_date - STATS_LAG
print(idx, st, min_date, max_date, stats_max_date, max_lag)

In [ ]:
both = all_stats.loc[st, :].loc['2020-08-01':, :][['Daily', 'NewHosp']].copy()
both.tail(60)

In [ ]:
both = all_stats.loc[st, :].loc['2020-08-01':, :][['Daily', 'NewHosp']].copy()
both.columns = ['Deaths', 'Hospital']
fam = both.plot(
    title=f"New Hospitalizations vs. Daily Deaths",
    secondary_y='Deaths', figsize=(16,5), ylim=0)
__ = fam.axvline(min_date, color="red", linestyle="--")
__ = fam.axvline(stats_max_date, color="red", linestyle="--")
__ = fam.axvline(max_date, color="green", linestyle="--")
__ = fam.get_figure().get_axes()[1].set_ylim(0)

best, best_sh, best_corr = None, 0.0, 0.0
for shift in range(MIN_SHIFT-4, MAX_SHIFT+4):
    h = both.Hospital.shift(shift).loc[min_date:stats_max_date]
    d = both.Deaths.loc[min_date:stats_max_date]
    corr = d.corr(h)
    print(shift, corr)
    if corr > best_corr:
        best_sh, best_corr = shift, corr
        best = pandas.concat([d, h], axis=1)
print("Best:", best_sh, best_corr)
if best_sh < MIN_SHIFT:
    best_sh = MIN_SHIFT
    print(f"Pinning shift to {MIN_SHIFT}")
if best_sh > MAX_SHIFT:
    best_sh = MAX_SHIFT
    print(f"Pinning shift to {MAX_SHIFT}")

# fam = best.plot(
#     title=f"Current Hospitalizations vs. Daily Deaths",
#     secondary_y='Deaths', figsize=(16,5), ylim=0)
# __ = fam.get_figure().get_axes()[1].set_ylim(0)

# h = both.Hospital.shift(best_sh).loc[max_date-RATIO_DAYS:max_date].sum()
h = both.Hospital.shift(hosp_lag).loc[max_date-RATIO_DAYS:max_date].sum()
d = both.Deaths.loc[max_date-RATIO_DAYS:max_date].sum()
hd_ratio = h / d

proj = both.copy()
proj.Hospital = proj.Hospital.shift(hosp_lag)
proj['Projected'] = proj.Deaths

old_vals = proj.Projected.loc[max_date:]
new_vals = proj.Hospital.loc[max_date:] / hd_ratio
proj.Projected.loc[max_date:] = new_vals.combine(old_vals, max)

print(f"proj={proj.Projected.sum()}, deaths={proj.Deaths.sum()}")

fam = proj.plot(
    title=f"Current Hospitalizations vs. Daily Deaths",
    secondary_y='Hospital', figsize=(16,5), ylim=0)
__ = fam.get_figure().get_axes()[1].set_ylim(0)

In [ ]:
both.loc['2021-01-02':'2021-01-10', :].Deaths.sum()

In [ ]:
spaz = both.copy()
spaz.loc['2021-01-02':'2021-01-10', 'Deaths'] += 17
fam = spaz.plot(
    title=f"Current Hospitalizations vs. Daily Deaths",
    secondary_y='Hospital', figsize=(16,5), ylim=0)

In [ ]:
both.Deaths.sum()

In [ ]:
raise ValueError()

In [ ]:
MAX_SHIFT = 7
for st, orig_shift, min_date, max_lag in ST_STATS:
    min_date, max_date = pandas.Period(min_date, freq='D'), LATEST_DATE - max_lag
    stats_max_date = max_date - STATS_LAG
    both = STATES[st]

    best, best_sh, best_corr = None, 0.0, 0.0
    for shift in range(-MAX_SHIFT, MAX_SHIFT+1):
        h = both.Hospital.shift(shift).loc[min_date:stats_max_date]
        d = both.Deaths.loc[min_date:stats_max_date]
        corr = d.corr(h)
        # print(shift, corr)
        if corr > best_corr:
            best_sh, best_corr = shift, corr
            best = pandas.concat([d, h], axis=1)
    print(f"{st}: {best_sh}, {best_corr}")
#     if best_sh < MIN_SHIFT:
#         best_sh = MIN_SHIFT
#         print(f"Pinning shift to {MIN_SHIFT}")
#     if best_sh > MAX_SHIFT:
#         best_sh = MAX_SHIFT
#         print(f"Pinning shift to {MAX_SHIFT}")